In [51]:
import pandas as pd
import numpy as np
import openpyxl

temp = pd.read_excel('data/template_2018이전.xlsx', engine='openpyxl')
data_2013 = pd.read_excel('data/실업급여_지급현황(월)_2013.xlsx', engine='openpyxl')

In [52]:
new_temp = temp[['년/월', '시도', '산업_대분류', '실업급여 수급자 수']]
new_temp.head()

,년/월,시도,산업_대분류,실업급여 수급자 수
0,201301,서울,제조업,NaN
1,201301,서울,도매 및 소매업,NaN
2,201301,서울,정보통신업,NaN
3,201301,서울,"전문, 과학 및 기술 서비스업",NaN
4,201301,서울,"사업시설 관리, 사업 지원 및 임대 서비스업",NaN


In [53]:
data_2013.head()

,마감년월,(거주지)시도,산업_대분류,직종_중분류,실업급여지급자수(월)
0,2013년01월,서울특별시,"09차_농업, 임업 및 어업",2005직종_관리직,10
1,NaN,NaN,NaN,2007직종_관리직,22
2,NaN,NaN,NaN,"2005직종_경영,회계,사무관련직",7
3,NaN,NaN,NaN,"2007직종_경영,회계,사무관련직",6
4,NaN,NaN,NaN,2005직종_영업및판매관련직,3


In [71]:
def pre_df(df, temp):
    df.fillna(method='pad', inplace=True)
    df['직종_중분류'] = df['직종_중분류'].str.replace('2005직종_', '', regex=True).replace('2007직종_', '', regex=True).replace('2018직종_', '', regex=True)
    df['산업_대분류'] = df['산업_대분류'].str.replace('09차_', '', regex=True).replace('10차_', '', regex=True)
    df.loc[df['산업_대분류'] == '출판, 영상, 방송통신 및 정보서비스업', '산업_대분류'] = '정보통신업'
    df.loc[df['산업_대분류'] == '사업시설관리 및 사업지원 서비스업', '산업_대분류'] = '사업시설 관리, 사업 지원 및 임대 서비스업'
    df['마감년월'] = df['마감년월'].str.replace('년', '', regex=True).replace('월', '', regex=True)
    target = df.loc[(df['직종_중분류'] == '정보통신 관련직') | (df['직종_중분류'] == '정보통신 연구개발직 및 공학기술직')].reset_index(drop=True)
    del target['직종_중분류']
    target = target[target['(거주지)시도'] != '분류불능']
    region = {'서울특별시':'서울', '부산광역시':'부산', '인천광역시':'인천', '광주광역시':'광주', '대구광역시':'대구', '대전광역시':'대전',
              '울산광역시':'울산', '세종특별자치시':'세종', '경기도':'경기', '강원특별자치도':'강원', '충청남도':'충남', '충청북도':'충북',
              '전라남도':'전남', '전라북도':'전북', '경상남도':'경남', '경상북도':'경북', '제주도':'제주'}
    for k, v in region.items():
        target['(거주지)시도'] = target['(거주지)시도'].str.replace(k, v)
    
    target.columns = temp.columns
    target = target.groupby(['년/월', '시도', '산업_대분류'], sort=False).sum().reset_index()
    return target

def merge_file(year):
    import pandas as pd
    import numpy as np
    import openpyxl
    
    df = pd.read_excel(f'data/실업급여_지급현황(월)_{year}.xlsx', engine='openpyxl')
    temp = pd.read_excel('data/template_2018이전.xlsx', engine='openpyxl')
    temp = temp[['년/월', '시도', '산업_대분류', '실업급여 수급자 수']]

    import warnings
    warnings.filterwarnings('ignore')

    df = pre_df(df, temp)
    df.columns = temp.columns

    temp['년/월'] = temp['년/월'].astype('str')
    temp['실업급여 수급자 수'].fillna(0, inplace=True)

    temp['실업급여 수급자 수'] = temp['실업급여 수급자 수'].astype('int64')

    temp_year = temp.loc[temp['년/월'].str.startswith(f'{year}')]

    merged = pd.merge(temp_year, df, how='left', on=['년/월', '시도', '산업_대분류'])

    merged.drop(columns=['실업급여 수급자 수_x'], inplace=True)

    merged.columns = ['년/월', '시도', '산업_대분류', '실업급여 수급자 수']

    merged.fillna(0, inplace=True)

    # merged.to_csv(f'data/구인구직취업현황_{year}_merged.csv', index=False)
    
    return merged

In [55]:
df_2013 = merge_file(2013)
df_2014 = merge_file(2014)
df_2015 = merge_file(2015)
df_2016 = merge_file(2016)
df_2017 = merge_file(2017)

dfs = pd.concat([df_2013, df_2014, df_2015, df_2016, df_2017], axis=0, ignore_index=True)
print(len(new_temp), len(dfs))

5100 5100


In [56]:
dfs

,년/월,시도,산업_대분류,실업급여 수급자 수
0,201301,서울,제조업,97.0
1,201301,서울,도매 및 소매업,202.0
2,201301,서울,정보통신업,1230.0
3,201301,서울,"전문, 과학 및 기술 서비스업",70.0
4,201301,서울,"사업시설 관리, 사업 지원 및 임대 서비스업",62.0
...,...,...,...,...
5095,201712,제주,제조업,0.0
5096,201712,제주,도매 및 소매업,0.0
5097,201712,제주,정보통신업,11.0
5098,201712,제주,"전문, 과학 및 기술 서비스업",1.0


In [58]:
dfs.to_csv('data/실업급여수급자수_merge_2018이전.csv', index=False, encoding='utf-8')

- 2018년 이후 데이터도 오류가 있어 다시 수정

In [72]:
def merge_file_new(year):
    import pandas as pd
    import numpy as np
    import openpyxl
    
    df = pd.read_excel(f'data/실업급여_지급현황(월)_{year}.xlsx', engine='openpyxl')
    temp = pd.read_excel('data/template_2018이전.xlsx', engine='openpyxl')
    temp = temp[['년/월', '시도', '산업_대분류', '실업급여 수급자 수']]
    
    import warnings
    warnings.filterwarnings('ignore')

    df = pre_df(df, temp)
    df.columns = temp.columns
    

    temp['년/월'] = temp['년/월'].astype('str')
    temp['년/월'] = temp['년/월'].str.replace('2013', '2018', regex=True)
    temp['년/월'] = temp['년/월'].str.replace('2014', '2019', regex=True)
    temp['년/월'] = temp['년/월'].str.replace('2015', '2020', regex=True)
    temp['년/월'] = temp['년/월'].str.replace('2016', '2021', regex=True)
    temp['년/월'] = temp['년/월'].str.replace('2017', '2022', regex=True)
    temp['실업급여 수급자 수'].fillna(0, inplace=True)

    temp['실업급여 수급자 수'] = temp['실업급여 수급자 수'].astype('int64')

    temp_year = temp.loc[temp['년/월'].str.startswith(f'{year}')]

    merged = pd.merge(temp_year, df, how='left', on=['년/월', '시도', '산업_대분류'])

    merged.drop(columns=['실업급여 수급자 수_x'], inplace=True)

    merged.columns = ['년/월', '시도', '산업_대분류', '실업급여 수급자 수']

    merged.fillna(0, inplace=True)

    # merged.to_csv(f'data/구인구직취업현황_{year}_merged.csv', index=False)
    
    return merged

In [73]:
df_2018 = merge_file_new(2018)
df_2019 = merge_file_new(2019)
df_2020 = merge_file_new(2020)
df_2021 = merge_file_new(2021)
df_2022 = merge_file_new(2022)

dfs = pd.concat([df_2018, df_2019, df_2020, df_2021, df_2022], axis=0, ignore_index=True)
print(len(new_temp), len(dfs))

5100 5100


In [74]:
dfs

,년/월,시도,산업_대분류,실업급여 수급자 수
0,201801,서울,제조업,104.0
1,201801,서울,도매 및 소매업,128.0
2,201801,서울,정보통신업,1042.0
3,201801,서울,"전문, 과학 및 기술 서비스업",94.0
4,201801,서울,"사업시설 관리, 사업 지원 및 임대 서비스업",63.0
...,...,...,...,...
5095,202212,제주,제조업,2.0
5096,202212,제주,도매 및 소매업,6.0
5097,202212,제주,정보통신업,23.0
5098,202212,제주,"전문, 과학 및 기술 서비스업",4.0


In [75]:
dfs.to_csv('data/실업급여수급자수_merge.csv', index=False, encoding='utf-8')